# Tarefa 1 — Bagging (Bootstrap Aggregating)

## Passo a passo do método

* <b>Passo 1</b>   — Separar um conjunto de treino
O Bagging parte de um conjunto de dados de treino, que será a base para gerar diferentes amostras.

* <b>Passo 2</b> — Bootstrap (gerar várias amostras com reposição)
A partir do treino, são criadas várias amostras bootstrap. Cada amostra é formada sorteando observações com reposição, o que faz com que algumas linhas possam aparecer repetidas e outras possam ficar de fora naquela amostra específica.

* <b>Passo 3</b> — Treinar vários modelos base (base learners)
Para cada amostra bootstrap gerada, é treinado um modelo base. No final, existe um conjunto de modelos (Modelo 1, Modelo 2, Modelo 3, etc.), cada um treinado em uma amostra ligeiramente diferente.

* <b>Passo 4</b> — Fazer previsões com todos os modelos
Quando chega um novo dado (ou o conjunto de teste), esse dado é passado por todos os modelos treinados, gerando várias previsões para a mesma observação.

* <b>Passo 5</b> — Aggregating (agregar/combinar as previsões)
As previsões individuais são combinadas em uma única saída final. Em classificação, a combinação normalmente é feita por votação (vence a classe mais frequente). Em regressão, a combinação é feita pela média das previsões numéricas.

* <b>Passo 6</b> — Obter a previsão final do ensemble
Depois da agregação, o resultado passa a ser a previsão final do Bagging, que representa a decisão do conjunto de modelos.


## O que é Bagging?

Bagging é uma técnica de ensemble que cria vários modelos do mesmo tipo e combina as previsões deles para gerar um resultado final mais estável e confiável. A ideia principal é que, em vez de depender de um único modelo (que pode variar bastante conforme o conjunto de treino muda), o Bagging usa um “conjunto” de modelos e toma uma decisão conjunta.

O processo funciona assim: primeiro são geradas várias amostras do conjunto de treino usando bootstrap, isto é, amostragem com reposição. Isso faz com que cada amostra fique um pouco diferente, porque algumas observações podem aparecer repetidas e outras podem não aparecer naquela amostra específica. Em seguida, um modelo base é treinado em cada amostra bootstrap. Depois, quando se faz uma previsão, cada modelo gera sua resposta e essas respostas são combinadas. Em classificação, a combinação costuma ser feita por votação (vence a classe mais frequente). Em regressão, a combinação geralmente é feita pela média das previsões numéricas.

## Exemplo do modelo

In [10]:
import numpy as np
import pandas as pd
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [11]:
data = load_breast_cancer(as_frame=True)
df = data.frame.copy()  # já vem com a coluna alvo "target"

print("Shape do df:", df.shape)
print("Colunas:", df.columns.tolist())
df.head()


Shape do df: (569, 31)
Colunas: ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension', 'target']


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [16]:
# Definindo X e y a partir do df
target = "target"
X = df.drop(columns=[target])
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

n_estimators = 100
rng = np.random.RandomState(42)

modelos = []
n_train = len(X_train)


# 1) BOOTSTRAP + 2) MODELAGEM

for _ in range(n_estimators):
    # BOOTSTRAP: sorteia índices com reposição
    idx = rng.choice(n_train, size=n_train, replace=True)
    X_boot = X_train.iloc[idx]
    y_boot = y_train.iloc[idx]

    # MODELAGEM: treina um modelo base nessa amostra bootstrap
    modelo = DecisionTreeClassifier(random_state=rng.randint(0, 10_000))
    modelo.fit(X_boot, y_boot)
    modelos.append(modelo)

# 3) AGREGAÇÃO: voto majoritário

pred_matrix = np.vstack([m.predict(X_test) for m in modelos])  # (n_estimators, n_test)

y_pred = []
for j in range(pred_matrix.shape[1]):
    valores, contagens = np.unique(pred_matrix[:, j], return_counts=True)
    y_pred.append(valores[np.argmax(contagens)])
y_pred = np.array(y_pred)

acc = accuracy_score(y_test, y_pred)

res = pd.DataFrame({
    "y_real": y_test.reset_index(drop=True),
    "y_pred": pd.Series(y_pred)
})

print(f"Acurácia (Bagging manual): {acc:.4f}")
print("Linhas no resumo:", len(res))
res.head()


Acurácia (Bagging manual): 0.9415
Linhas no resumo: 171


,y_real,y_pred
0,0,0
1,1,1
2,1,1
3,0,0
4,0,1
